# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [219]:
import pandas as pd
import numpy as numpy
from scipy import stats
import matplotlib.pyplot as pyplot
numpy.random.seed(17)

# some nice colors from http://colorbrewer2.org/
COLOR1 = '#7fc97f'
COLOR2 = '#beaed4'
COLOR3 = '#fdc086'
COLOR4 = '#ffff99'
COLOR5 = '#386cb0'

%matplotlib inline


In [220]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [221]:
# number of callbacks for black-sounding names
# Call back ratios for white and black sounding names

sum(data[data.race=='w'].call)/len(data[data.race=='w'])
sum(data[data.race=='b'].call)/len(data[data.race=='b'])

0.064476386036960986

In [222]:
data.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [223]:
w = data[data.race=='w']
b = data[data.race=='b']

In [224]:
class Resampler(object):
    """Represents a framework for computing sampling distributions."""
    
    def __init__(self, sample, xlim=None):
        """Stores the actual sample."""
        self.sample = sample
        self.n = 200
        self.xlim = xlim
        
    def resample(self):
        """Generates a new sample by choosing from the original
        sample with replacement.
        """
        new_sample = numpy.random.choice(self.sample, self.n, replace=False)
        return new_sample
    
    def sample_stat(self, sample):
        """Computes a sample statistic using the original sample or a
        simulated sample.
        """
        return sample.mean()
        
    
    def compute_sampling_distribution(self, iters=1000):
        """Simulates many experiments and collects the resulting sample
        statistics.
        """
        stats = [self.sample_stat(self.resample()) for i in range(iters)]
        return numpy.array(stats)
    
    def plot_sampling_distribution(self):
        """Plots the sampling distribution."""
        sample_stats = self.compute_sampling_distribution()
        se = sample_stats.std()
        ci = numpy.percentile(sample_stats, [5, 95])
        me= sample_stats.mean()
        print (ci)
        pyplot.hist(sample_stats, color=COLOR2)
        pyplot.xlabel('sample statistic')
        pyplot.xlim(self.xlim)
        pyplot.text(0.03, 0.95, 'CI [%0.2f %0.2f]' % tuple(ci))
        pyplot.text(0.03, 0.85, 'SE %0.2f' % se)
        pyplot.text(0.03, 0.75, 'mean %0.2f' % me)
        pyplot.show()

In [225]:
white_array=numpy.array(w.call)
black_array=numpy.array(b.call)

black_array_object=Resampler(black_array)
white_array_object=Resampler(white_array)

In [226]:
black_mean=black_array_object.compute_sampling_distribution().mean()
white_mean=white_array_object.compute_sampling_distribution().mean()
black_std=black_array_object.compute_sampling_distribution().std()
white_std=white_array_object.compute_sampling_distribution().std()

In [227]:
diff_mean=white_mean-black_mean

In [228]:
diff_std= np.sqrt((np.square(black_std)+np.square(white_std))/2)

In [229]:
diff_std , black_std , white_std

(0.018425092045269889, 0.016427206, 0.02022659)

### Confidence Interval and Margin of error

#### For 95% condidence interval calculating margin of error

In [230]:
Margin_error=stats.norm.ppf(.975)

In [231]:
CI = diff_mean - (diff_std * Margin_error) , diff_mean + (diff_std * Margin_error)

In [232]:
CI

(-0.0045474988223458898, 0.06767753481878297)

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

**  Lets setup the hypothesis **  
    Null hypothesis: Difference in means=0 (no discrimination) ;
    Alternate hypothesis: Difference in means > 0  
    Lets set the threshold at 5% 

### Z Score

In [237]:
z_Score=(diff_mean - 0)/diff_std

In [234]:
z_Score

1.7131538838809732

In [236]:
P_value = 1-stats.norm.cdf(z_Score)
P_value

0.04334211655784681

### The widely accepted threshold is 5%. As P-value is lower than threshold we cannot accept the Null hypothesis in this particular case. Therefore this means that race based discrimination is a possibility.
